In [ ]:
# =======================
#  CELL 1
#  Step 1: Data Fetching & Setup
# =======================

import sys

import datetime
import random
import numpy as np
import pandas as pd




# Set random seed for reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

try:
    import yfinance as yf
except ImportError:
    print("Please install yfinance (pip install yfinance) before proceeding.")
    # In Colab, you can do: !pip install yfinance
    raise

def fetch_data_from_yahoo(tickers, start_date, end_date):
    """
    Fetch Adjusted Close data from Yahoo Finance for given tickers and date range.

    Parameters:
    -----------
    - tickers: list of str
    - start_date: datetime or str
    - end_date: datetime or str

    Returns:
    --------
    - pandas.DataFrame: Adjusted Close prices, rows are dates, columns are tickers
    """
    data = yf.download(tickers, start=start_date, end=end_date)['Close']
    # Forward-fill any missing data, then drop remaining NAs
    data.ffill(inplace=True)
    data.dropna(inplace=True)
    return data

# Example Tickers for top 20 NSE
nse_tickers = [
    'RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS',
    'HINDUNILVR.NS', 'KOTAKBANK.NS', 'LT.NS', 'SBIN.NS', 'AXISBANK.NS',
    'HCLTECH.NS', 'BAJFINANCE.NS', 'ITC.NS', 'BHARTIARTL.NS', 'ASIANPAINT.NS',
    'MARUTI.NS', 'SUNPHARMA.NS', 'TITAN.NS', 'WIPRO.NS', 'ULTRACEMCO.NS'
]

# Fetch Data
end_date = datetime.datetime.today()
start_date = end_date - datetime.timedelta(days=720)  # ~2 years

print("Fetching sample data from Yahoo Finance in Colab environment...")
try:
    price_data = fetch_data_from_yahoo(nse_tickers, start_date, end_date)
    print(f"Data fetched successfully! Shape: {price_data.shape}")
    print(f"Date Range: {price_data.index.min()} to {price_data.index.max()}")
    print(f"First few rows:\n{price_data.head()}")
except Exception as e:
    print(f"Error fetching data: {e}")
    price_data = None

print("STEP 1 completed in Colab cell!")


In [ ]:
# =======================
#  CELL 2 - Revised
#  Step 2: Classic ML Models (Per-Asset Neural Network and XGBoost)
# =======================


import numpy as np
import pandas as pd
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from scikeras.wrappers import KerasRegressor  # Correct import for scikeras

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error

import xgboost as xgb
from math import sqrt
import joblib  # For saving/loading models

SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

###############################################
# 1. Extended Feature Engineering
###############################################

def compute_daily_returns(prices_df):
    """Compute daily returns from price data."""
    return prices_df.pct_change().dropna()

def compute_rolling_volatility(returns_df, window=20):
    """Compute rolling volatility over a given window."""
    return returns_df.rolling(window).std().dropna()

def compute_ma(prices_df, window=20):
    """Compute moving average."""
    return prices_df.rolling(window).mean().dropna()

def compute_rsi(prices_df, window=14):
    """Compute RSI for each column in prices_df."""
    def _single_rsi(series):
        delta = series.diff().dropna()
        gain = delta.where(delta > 0, 0).rolling(window=window).mean()
        loss = -delta.where(delta < 0, 0).rolling(window=window).mean()
        rs = gain / (loss + 1e-15)
        return 100 - (100 / (1 + rs))
    return prices_df.apply(_single_rsi).dropna()

def build_feature_matrix(prices_df):
    """
    Combine daily returns, rolling volatility, 20-day MA, and RSI(14).
    Shift next-day returns as the target.
    """
    # 1) Daily returns
    daily_ret = compute_daily_returns(prices_df)

    # 2) Rolling volatility on daily returns
    vol = compute_rolling_volatility(daily_ret, 20)

    # 3) 20-day Moving Average
    ma20 = compute_ma(prices_df, 20)

    # 4) RSI(14)
    rsi = compute_rsi(prices_df, 14)

    # Combine all features
    combined = pd.concat([
        daily_ret.add_suffix('_Ret'),
        vol.add_suffix('_Vol'),
        ma20.add_suffix('_MA20'),
        rsi.add_suffix('_RSI')
    ], axis=1).dropna()

    # Shift next-day returns as target
    shifted = daily_ret.shift(-1).dropna()

    # Align the feature matrix with the shifted targets
    combined = combined.iloc[:-1]  # Drop the last row to match the shifted target
    combined.dropna(inplace=True)
    shifted.dropna(inplace=True)

    # Final alignment
    X, y = combined.align(shifted, join='inner', axis=0)
    return X, y

###############################################
# 2. Train Per-Asset Neural Network Models
###############################################

def create_keras_model(input_dim=24, output_dim=1, n_hidden=1, n_neurons=32, dropout=0.2, lr=0.001):
    """Build a Keras model based on given hyperparameters."""
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_dim,)))
    for _ in range(n_hidden):
        model.add(layers.Dense(n_neurons, activation='relu'))
        model.add(layers.Dropout(dropout))
    model.add(layers.Dense(output_dim, activation='linear'))

    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='mse')
    return model

def tune_neural_network(X_train, y_train):
    """
    Tune a Neural Network using scikeras's KerasRegressor and RandomizedSearchCV.
    """
    input_dim = X_train.shape[1]
    output_dim = 1  # Per asset

    # Create the KerasRegressor with a fixed input/output shape
    keras_reg = KerasRegressor(
        model=create_keras_model,
        input_dim=input_dim,  # Fixed input dimension
        output_dim=output_dim,  # Fixed output dimension
        verbose=0
    )

    # Define the hyperparameter grid
    param_distributions = {
        'model__n_hidden': [1, 2],
        'model__n_neurons': [32, 64],
        'model__dropout': [0.2, 0.3],
        'model__lr': [1e-3, 1e-4],
        'epochs': [30, 50],
        'batch_size': [32, 64]
    }

    # Define TimeSeriesSplit for cross-validation
    tscv = TimeSeriesSplit(n_splits=3)

    # Initialize RandomizedSearchCV
    search = RandomizedSearchCV(
        estimator=keras_reg,
        param_distributions=param_distributions,
        n_iter=6,  # Increase for more thorough search
        cv=tscv,
        scoring='neg_mean_squared_error',
        random_state=SEED,
        verbose=0
    )

    # Fit the model
    search.fit(X_train, y_train)

    print(f"Best NN params= {search.best_params_}")
    best_model = search.best_estimator_
    return best_model

def train_neural_network_per_asset(X_train, X_test, y_train, y_test):
    """
    Train Neural Network models for each asset's DeltaR and DeltaSigma.

    Returns:
    --------
    - dict: {'DeltaR': {asset: model, ...}, 'DeltaSigma': {asset: model, ...}}
    """
    model_dict = {'DeltaR': {}, 'DeltaSigma': {}}
    assets = y_train.columns

    for asset in assets:
        print(f"Training Neural Network for DeltaR of asset: {asset}")
        # Train model for DeltaR
        nn_model_r = tune_neural_network(X_train, y_train[asset])
        model_dict['DeltaR'][asset] = nn_model_r

        print(f"Training Neural Network for DeltaSigma of asset: {asset}")
        # Train model for DeltaSigma
        nn_model_s = tune_neural_network(X_train, y_train[asset])
        model_dict['DeltaSigma'][asset] = nn_model_s

    return model_dict

def evaluate_neural_networks_per_asset(model_dict, X, y):
    """
    Evaluate the Neural Network models for each asset's DeltaR and DeltaSigma.

    Returns:
    --------
    - dict: {'DeltaR': {asset: rmse, ...}, 'DeltaSigma': {asset: rmse, ...}}
    """
    rmse_dict = {'DeltaR': {}, 'DeltaSigma': {}}
    assets = y.columns

    for asset in assets:
        # Evaluate DeltaR
        pred_r = model_dict['DeltaR'][asset].predict(X)
        rmse_r = sqrt(mean_squared_error(y[asset], pred_r))
        rmse_dict['DeltaR'][asset] = rmse_r

        # Evaluate DeltaSigma
        pred_s = model_dict['DeltaSigma'][asset].predict(X)
        rmse_s = sqrt(mean_squared_error(y[asset], pred_s))
        rmse_dict['DeltaSigma'][asset] = rmse_s

        print(f"[NeuralNet/{asset}/DeltaR] RMSE= {rmse_r:.5f}")
        print(f"[NeuralNet/{asset}/DeltaSigma] RMSE= {rmse_s:.5f}")

    return rmse_dict

def example_usage_neural_network_per_asset(X_train, X_test, y_train, y_test):
    """
    Full demonstration:
      1) Train Neural Network models for each asset's DeltaR and DeltaSigma.
      2) Evaluate on training and testing data.

    Returns:
    --------
    - dict: {'DeltaR': {asset: model, ...}, 'DeltaSigma': {asset: model, ...}}
    """
    # 1) Train models
    model_dict = train_neural_network_per_asset(X_train, X_test, y_train, y_test)

    # 2) Evaluate on training data
    print("Evaluating Neural Network models on training data...")
    train_rmse = evaluate_neural_networks_per_asset(model_dict, X_train, y_train)

    # 3) Evaluate on testing data
    print("Evaluating Neural Network models on testing data...")
    test_rmse = evaluate_neural_networks_per_asset(model_dict, X_test, y_test)

    # Summary
    print("=== Summary of Neural Network Models ===")
    for asset in y_train.columns:
        print(f"Asset: {asset}")
        print(f"  DeltaR Test RMSE: {test_rmse['DeltaR'][asset]:.5f}")
        print(f"  DeltaSigma Test RMSE: {test_rmse['DeltaSigma'][asset]:.5f}")

    return model_dict

###############################################
# 3. Train Per-Asset XGBoost Models
###############################################

def train_xgboost_models_per_asset(X_train, y_train):
    """
    Train separate XGBoost models for each asset's DeltaR and DeltaSigma.

    Returns:
    --------
    - dict: {'DeltaR': {asset: model, ...}, 'DeltaSigma': {asset: model, ...}}
    """
    model_dict = {'DeltaR': {}, 'DeltaSigma': {}}
    assets = y_train.columns
    tscv = TimeSeriesSplit(n_splits=3)

    for asset in assets:
        print(f"Training XGBoost model for DeltaR of asset: {asset}")
        # Train model for DeltaR
        xgb_model_r = xgb.XGBRegressor(objective='reg:squarederror', random_state=SEED)
        param_grid_r = {
            'n_estimators': [100, 200],
            'max_depth': [3, 5],
            'learning_rate': [0.01, 0.1]
        }
        grid_r = GridSearchCV(
            estimator=xgb_model_r,
            param_grid=param_grid_r,
            cv=tscv,
            scoring='neg_mean_squared_error',
            verbose=0
        )
        grid_r.fit(X_train, y_train[asset])
        print(f"Best Params for DeltaR ({asset}): {grid_r.best_params_}")
        model_dict['DeltaR'][asset] = grid_r.best_estimator_

        print(f"Training XGBoost model for DeltaSigma of asset: {asset}")
        # Train model for DeltaSigma
        xgb_model_s = xgb.XGBRegressor(objective='reg:squarederror', random_state=SEED)
        param_grid_s = {
            'n_estimators': [100, 200],
            'max_depth': [3, 5],
            'learning_rate': [0.01, 0.1]
        }
        grid_s = GridSearchCV(
            estimator=xgb_model_s,
            param_grid=param_grid_s,
            cv=tscv,
            scoring='neg_mean_squared_error',
            verbose=0
        )
        grid_s.fit(X_train, y_train[asset])
        print(f"Best Params for DeltaSigma ({asset}): {grid_s.best_params_}")
        model_dict['DeltaSigma'][asset] = grid_s.best_estimator_

    return model_dict

def evaluate_xgboost_models_per_asset(model_dict, X, y):
    """
    Evaluate the XGBoost models for each asset's DeltaR and DeltaSigma.

    Returns:
    --------
    - dict: {'DeltaR': {asset: rmse, ...}, 'DeltaSigma': {asset: rmse, ...}}
    """
    rmse_dict = {'DeltaR': {}, 'DeltaSigma': {}}
    assets = y.columns

    for asset in assets:
        # Evaluate DeltaR
        pred_r = model_dict['DeltaR'][asset].predict(X)
        rmse_r = sqrt(mean_squared_error(y[asset], pred_r))
        rmse_dict['DeltaR'][asset] = rmse_r

        # Evaluate DeltaSigma
        pred_s = model_dict['DeltaSigma'][asset].predict(X)
        rmse_s = sqrt(mean_squared_error(y[asset], pred_s))
        rmse_dict['DeltaSigma'][asset] = rmse_s

        print(f"[XGBoost/{asset}/DeltaR] RMSE= {rmse_r:.5f}")
        print(f"[XGBoost/{asset}/DeltaSigma] RMSE= {rmse_s:.5f}")

    return rmse_dict

def example_usage_xgboost_per_asset(X_train, X_test, y_train, y_test):
    """
    Full demonstration:
      1) Train XGBoost models for each asset's DeltaR and DeltaSigma.
      2) Evaluate on training and testing data.

    Returns:
    --------
    - dict: {'DeltaR': {asset: model, ...}, 'DeltaSigma': {asset: model, ...}}
    """
    # 1) Train models
    model_dict = train_xgboost_models_per_asset(X_train, y_train)

    # 2) Evaluate on training data
    print("Evaluating XGBoost models on training data...")
    train_rmse = evaluate_xgboost_models_per_asset(model_dict, X_train, y_train)

    # 3) Evaluate on testing data
    print("Evaluating XGBoost models on testing data...")
    test_rmse = evaluate_xgboost_models_per_asset(model_dict, X_test, y_test)

    # Summary
    print("=== Summary of XGBoost Models ===")
    for asset in y_train.columns:
        print(f"Asset: {asset}")
        print(f"  DeltaR Test RMSE: {test_rmse['DeltaR'][asset]:.5f}")
        print(f"  DeltaSigma Test RMSE: {test_rmse['DeltaSigma'][asset]:.5f}")

    return model_dict

###############################################
# 4. Main Step 2 Logic with Extended Features and Hyperparameter Tuning
###############################################

if 'price_data' not in globals() or price_data is None:
    print("price_data not found. Please run the Step 1 cell first!")
else:
    print("Building extended feature matrix & shifted target (next-day returns)...")
    X, y = build_feature_matrix(price_data)
    print(f"Feature matrix shape= {X.shape}, target shape= {y.shape}")

    # Train/Test Split
    def time_series_split_data(X, y, test_size=0.2):
        """
        Split data into train and test sets without shuffling.
        """
        n = len(X)
        split_idx = int(n * (1 - test_size))
        X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
        y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
        return X_train, X_test, y_train, y_test

    X_train, X_test, y_train, y_test = time_series_split_data(X, y, test_size=0.2)
    print(f"X_train= {X_train.shape}, X_test= {X_test.shape}")

    # Train Neural Network models per asset
    print("Training Neural Network models for each asset's DeltaR and DeltaSigma...")
    nn_models = example_usage_neural_network_per_asset(X_train, X_test, y_train, y_test)

    # Train XGBoost models per asset
    print("Training XGBoost models for each asset's DeltaR and DeltaSigma...")
    xgb_models = example_usage_xgboost_per_asset(X_train, X_test, y_train, y_test)

    print("STEP 2 completed (extended features, hyperparameter tuning for Neural Networks and XGBoost)!")


In [ ]:
# =======================
#  CELL 3 - Final Revised with Ensemble and Enhanced CV
#  Step 3: GNN Integration (Post-Installation)
# =======================

import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, SAGEConv, GATConv
import numpy as np
import pandas as pd
import dcor  # For distance correlation
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import TimeSeriesSplit

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler  # For target normalization

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

###############################################
# 1. Define GNN Models with Different Aggregators
###############################################

class MarginalGNN(nn.Module):
    """
    A 3-layer Graph Convolutional Network for predicting [DeltaR, DeltaSigma] per asset.
    Supports different aggregator types: 'gcn', 'sage', 'gat'.
    """
    def __init__(self, in_channels: int, hidden_channels: int = 64, out_channels: int = 2, gnn_type: str = 'gcn'):
        super(MarginalGNN, self).__init__()

        if gnn_type == 'gcn':
            self.conv1 = GCNConv(in_channels, hidden_channels)
            self.conv2 = GCNConv(hidden_channels, hidden_channels)
            self.conv3 = GCNConv(hidden_channels, out_channels)
        elif gnn_type == 'sage':
            self.conv1 = SAGEConv(in_channels, hidden_channels)
            self.conv2 = SAGEConv(hidden_channels, hidden_channels)
            self.conv3 = SAGEConv(hidden_channels, out_channels)
        elif gnn_type == 'gat':
            self.conv1 = GATConv(in_channels, hidden_channels, heads=2, dropout=0.6)
            self.conv2 = GATConv(hidden_channels * 2, hidden_channels, heads=2, dropout=0.6)
            self.conv3 = GATConv(hidden_channels * 2, out_channels, heads=1, concat=False, dropout=0.6)
        else:
            raise ValueError("Unsupported GNN type. Choose from 'gcn', 'sage', 'gat'.")

    def forward(self, x, edge_index, edge_attr=None):
        if isinstance(self.conv1, GATConv):
            x = self.conv1(x, edge_index)
        else:
            x = self.conv1(x, edge_index)
        x = torch.relu(x)

        if isinstance(self.conv2, GATConv):
            x = self.conv2(x, edge_index)
        else:
            x = self.conv2(x, edge_index)
        x = torch.relu(x)

        if isinstance(self.conv3, GATConv):
            x = self.conv3(x, edge_index)
        else:
            x = self.conv3(x, edge_index)
        return x  # Output shape: (n_assets, 2)

###############################################
# 2. Define Functions for Distance Correlation and Adjacency Matrix
###############################################

def calculate_distance_correlation(price_data: pd.DataFrame) -> np.ndarray:
    """
    Calculate the distance correlation matrix for all pairs of assets.
    """
    assets = price_data.columns
    n_assets = len(assets)
    dist_corr_matrix = np.zeros((n_assets, n_assets), dtype=np.float32)

    print("Calculating distance correlation matrix...")
    for i in range(n_assets):
        for j in range(i + 1, n_assets):
            dist_corr = dcor.distance_correlation(price_data.iloc[:, i], price_data.iloc[:, j])
            dist_corr_matrix[i, j] = dist_corr
            dist_corr_matrix[j, i] = dist_corr  # Symmetric matrix

    print("Distance correlation matrix calculation complete.")
    return dist_corr_matrix

def build_weighted_adjacency_matrix(dist_corr_matrix: np.ndarray, threshold: float) -> np.ndarray:
    """
    Build a weighted adjacency matrix based on distance correlation threshold.
    """
    print(f"Building weighted adjacency matrix with threshold={threshold}...")
    adjacency = np.where(dist_corr_matrix >= threshold, dist_corr_matrix, 0.0)
    np.fill_diagonal(adjacency, 0.0)  # Remove self-loops
    print("Weighted adjacency matrix construction complete.")
    return adjacency

def build_unweighted_adjacency_matrix(dist_corr_matrix: np.ndarray, threshold: float) -> np.ndarray:
    """
    Build an unweighted adjacency matrix based on distance correlation threshold.
    """
    print(f"Building unweighted adjacency matrix with threshold={threshold}...")
    adjacency = np.where(dist_corr_matrix >= threshold, 1.0, 0.0)
    np.fill_diagonal(adjacency, 0.0)  # Remove self-loops
    print("Unweighted adjacency matrix construction complete.")
    return adjacency

###############################################
# 3. Feature Engineering
###############################################

def engineer_node_features(price_data: pd.DataFrame) -> pd.DataFrame:
    """
    Engineer node features including average return, volatility, momentum, etc.
    """
    print("Engineering enhanced node features...")
    # Calculate daily returns
    returns = price_data.pct_change().dropna()

    # Feature 1: Average Historical Return
    avg_return = returns.mean().values.reshape(-1, 1)

    # Feature 2: Historical Volatility
    volatility = returns.std().values.reshape(-1, 1)

    # Feature 3: 5-Day Momentum
    momentum_5d = price_data.pct_change(periods=5).mean().values.reshape(-1, 1)

    # Feature 4: 10-Day Momentum
    momentum_10d = price_data.pct_change(periods=10).mean().values.reshape(-1, 1)

    # Feature 5: Maximum Drawdown (past 30 days)
    rolling_max = price_data.rolling(window=30).max()
    drawdown = (price_data / rolling_max) - 1.0
    max_drawdown = drawdown.min().values.reshape(-1, 1)

    # Feature 6: Sharpe Ratio (assuming risk-free rate = 0)
    sharpe_ratio = (returns.mean().values.reshape(-1,1)) / (returns.std().values.reshape(-1,1) + 1e-15)

    # Combine all features
    features = np.hstack((avg_return, volatility, momentum_5d, momentum_10d, max_drawdown, sharpe_ratio))
    feature_df = pd.DataFrame(features, columns=['avg_return', 'volatility', 'momentum_5d', 'momentum_10d', 'max_drawdown', 'sharpe_ratio'])

    # Normalize features
    feature_df = (feature_df - feature_df.mean()) / feature_df.std()

    print("Advanced feature engineering complete.")
    return feature_df

###############################################
# 4. Prepare Graph Data
###############################################

def prepare_graph_data(price_data: pd.DataFrame, deltaR: np.ndarray, deltaSigma: np.ndarray, threshold: float, use_weighted_adj: bool = True) -> Data:
    """
    Prepare the PyG Data object for GNN training.
    """
    # Calculate distance correlation matrix
    dist_corr_matrix = calculate_distance_correlation(price_data)

    # Build adjacency matrix based on threshold
    if use_weighted_adj:
        adjacency = build_weighted_adjacency_matrix(dist_corr_matrix, threshold)
    else:
        adjacency = build_unweighted_adjacency_matrix(dist_corr_matrix, threshold)

    # Feature Engineering
    node_features = engineer_node_features(price_data)

    # Prepare targets
    Y = np.column_stack((deltaR, deltaSigma))  # shape: (n_assets, 2)

    # Create edge_index
    edges = np.array(adjacency.nonzero())
    edge_index = torch.tensor(edges, dtype=torch.long)

    # Create PyG Data object
    data = Data(x=torch.tensor(node_features.values, dtype=torch.float),
                edge_index=edge_index,
                y=torch.tensor(Y, dtype=torch.float))

    return data

###############################################
# 5. Threshold Selection via Cross-Validation
###############################################

def cross_validate_threshold(price_data: pd.DataFrame, deltaR: np.ndarray, deltaSigma: np.ndarray, thresholds: list, gnn_type: str = 'gcn', hidden_channels: int = 32, epochs: int = 50, lr: float = 1e-3) -> float:
    """
    Perform cross-validation to select the optimal threshold for adjacency matrix.
    """
    print("Starting threshold selection via cross-validation...")
    tscv = TimeSeriesSplit(n_splits=3)
    n_assets = price_data.shape[1]
    asset_indices = np.arange(n_assets)

    threshold_rmse = {}

    for threshold in thresholds:
        print(f"Evaluating threshold: {threshold}")
        rmses = []

        for train_idx, val_idx in tscv.split(asset_indices):
            train_assets = asset_indices[train_idx]
            val_assets = asset_indices[val_idx]

            # Prepare training data
            train_price = price_data.iloc[:, train_assets]
            train_deltaR = deltaR[train_assets]
            train_deltaSigma = deltaSigma[train_assets]

            train_data = prepare_graph_data(train_price, train_deltaR, train_deltaSigma, threshold, use_weighted_adj=True)

            # Initialize GNN model
            model = MarginalGNN(in_channels=train_data.num_features, hidden_channels=hidden_channels, out_channels=2, gnn_type=gnn_type)

            # Train the model
            model = train_gnn(model, train_data, epochs=epochs, lr=lr)

            # Prepare validation data
            val_price = price_data.iloc[:, val_assets]
            val_deltaR = deltaR[val_assets]
            val_deltaSigma = deltaSigma[val_assets]

            val_data = prepare_graph_data(val_price, val_deltaR, val_deltaSigma, threshold, use_weighted_adj=True)

            # Evaluate the model on validation data
            rmse, _ = evaluate_gnn(model, val_data)
            rmses.append(rmse)

        avg_rmse = np.mean(rmses)
        threshold_rmse[threshold] = avg_rmse
        print(f"Threshold {threshold} => Average RMSE: {avg_rmse:.6f}")

    # Select the threshold with the lowest average RMSE
    optimal_threshold = min(threshold_rmse, key=threshold_rmse.get)
    print(f"Optimal threshold selected: {optimal_threshold} with RMSE: {threshold_rmse[optimal_threshold]:.6f}")
    return optimal_threshold

###############################################
# 6. Training and Evaluation Functions
###############################################

def train_gnn(model: nn.Module, data: Data, epochs: int = 100, lr: float = 1e-3) -> nn.Module:
    """
    Train the GNN model.

    Args:
        model (nn.Module): GNN model to train.
        data (torch_geometric.data.Data): Graph data.
        epochs (int): Number of training epochs.
        lr (float): Learning rate.

    Returns:
        nn.Module: Trained model.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    data = data.to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    model.train()
    for epoch in range(1, epochs + 1):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)  # shape: (n_assets, 2)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0 or epoch == 1:
            print(f"Epoch {epoch}/{epochs}, Loss: {loss.item():.6f}")

    return model

def evaluate_gnn(model: nn.Module, data: Data) -> tuple:
    """
    Evaluate the GNN model using RMSE.

    Args:
        model (nn.Module): Trained GNN model.
        data (torch_geometric.data.Data): Graph data.

    Returns:
        tuple: (RMSE score, Predicted marginal contributions as np.ndarray)
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    data = data.to(device)

    model.eval()
    with torch.no_grad():
        preds = model(data.x, data.edge_index)

    preds = preds.cpu().numpy()
    targets = data.y.cpu().numpy()

    rmse = sqrt(mean_squared_error(targets, preds))
    return rmse, preds

###############################################
# 7. Full GNN Workflow Integration Function
###############################################

def full_gnn_workflow(price_data: pd.DataFrame, deltaR: np.ndarray, deltaSigma: np.ndarray):
    """
    Complete workflow to train and evaluate GNNs with different aggregators.
    """
    # Step 1: Define a range of thresholds to evaluate
    thresholds = [0.2, 0.3, 0.4, 0.5]

    # Step 2: Perform cross-validation to select the optimal threshold
    optimal_threshold = cross_validate_threshold(price_data, deltaR, deltaSigma, thresholds, gnn_type='gcn', hidden_channels=32, epochs=50, lr=1e-3)

    # Step 3: Prepare the final graph data using the optimal threshold
    data = prepare_graph_data(price_data, deltaR, deltaSigma, optimal_threshold, use_weighted_adj=True)

    # Debugging: Check uniqueness of targets
    unique_deltaR = np.unique(data.y[:,0])
    unique_deltaSigma = np.unique(data.y[:,1])
    print(f"Unique DeltaR values: {unique_deltaR}")
    print(f"Unique DeltaSigma values: {unique_deltaSigma}")

    if len(unique_deltaR) == 1 or len(unique_deltaSigma) == 1:
        print("Warning: All target DeltaR or DeltaSigma values are identical. GNN may not learn effectively.")

    # Step 4: Define aggregator types to evaluate
    aggregators = ['gcn', 'sage', 'gat']

    # Step 5: Initialize a dictionary to store results
    results = {}

    # Step 6: Train and evaluate GNN models for each aggregator
    for agg in aggregators:
        print(f"Training GNN with aggregator: {agg.upper()}")

        # Initialize the model
        model = MarginalGNN(in_channels=data.num_features, hidden_channels=32, out_channels=2, gnn_type=agg)

        # Train the model
        trained_model = train_gnn(model, data, epochs=100, lr=1e-3)

        # Evaluate the model
        rmse, preds = evaluate_gnn(trained_model, data)

        # Store the results
        results[agg] = {
            'model': trained_model,
            'rmse': rmse,
            'predictions': preds
        }

        print(f"GNN with {agg.upper()} Aggregator => RMSE: {rmse:.6f}")

    # Step 7: Determine the best aggregator based on RMSE
    best_aggregator = min(results, key=lambda x: results[x]['rmse'])
    best_rmse = results[best_aggregator]['rmse']

    print(f"Best Aggregator: {best_aggregator.upper()} with RMSE: {best_rmse:.6f}")

    # Optional: Display predictions of the best model
    best_preds = results[best_aggregator]['predictions']
    pred_df = pd.DataFrame(best_preds, columns=['DeltaR_Pred', 'DeltaSigma_Pred'], index=price_data.columns)
    print("Predicted Marginal Contributions (Best Model):")
    print(pred_df)

    # Visualization: Actual vs Predicted
    plt.figure(figsize=(12,5))

    # DeltaR
    plt.subplot(1, 2, 1)
    plt.scatter(data.y[:,0], best_preds[:,0], color='blue')
    plt.xlabel('Actual DeltaR')
    plt.ylabel('Predicted DeltaR')
    plt.title(f'GNN Predictions vs Actual DeltaR ({best_aggregator.upper()})')
    plt.plot([data.y[:,0].min(), data.y[:,0].max()], [data.y[:,0].min(), data.y[:,0].max()], 'r--')

    # DeltaSigma
    plt.subplot(1, 2, 2)
    plt.scatter(data.y[:,1], best_preds[:,1], color='green')
    plt.xlabel('Actual DeltaSigma')
    plt.ylabel('Predicted DeltaSigma')
    plt.title(f'GNN Predictions vs Actual DeltaSigma ({best_aggregator.upper()})')
    plt.plot([data.y[:,1].min(), data.y[:,1].max()], [data.y[:,1].min(), data.y[:,1].max()], 'r--')

    plt.tight_layout()
    plt.show()

    # Visualization: Bar Charts
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.bar(pred_df.index, pred_df['DeltaR_Pred'], color='skyblue')
    plt.xlabel('Assets')
    plt.ylabel('DeltaR Predicted')
    plt.title('GNN Predicted DeltaR per Asset')
    plt.xticks(rotation=90)

    plt.subplot(1, 2, 2)
    plt.bar(pred_df.index, pred_df['DeltaSigma_Pred'], color='salmon')
    plt.xlabel('Assets')
    plt.ylabel('DeltaSigma Predicted')
    plt.title('GNN Predicted DeltaSigma per Asset')
    plt.xticks(rotation=90)

    plt.tight_layout()
    plt.show()

    return results

###############################################
# 8. Example Usage with Real Data
###############################################

# Ensure that 'xgb_models' and 'nn_models' dictionaries are available
if 'xgb_models' not in globals() or 'nn_models' not in globals():
    print("Classic ML models not found. Please run the Step 2 cell first!")
else:
    print("Generating DeltaR and DeltaSigma using trained Neural Network and XGBoost models...")
    n_assets = price_data.shape[1]

    # Initialize arrays to store predictions
    deltaR_final = np.zeros(n_assets)
    deltaSigma_final = np.zeros(n_assets)

    # Initialize lists to store RMSEs for dynamic weighting
    rmse_r_nn = []
    rmse_r_xgb = []
    rmse_s_nn = []
    rmse_s_xgb = []

    # Calculate RMSEs for weighting based on training data

    for asset in price_data.columns:
        # Neural Network RMSE for DeltaR
        nn_pred_r_train = nn_models['DeltaR'][asset].predict(X_train)
        rmse_r_nn.append(sqrt(mean_squared_error(y_train[asset], nn_pred_r_train)))

        # Neural Network RMSE for DeltaSigma
        nn_pred_s_train = nn_models['DeltaSigma'][asset].predict(X_train)
        rmse_s_nn.append(sqrt(mean_squared_error(y_train[asset], nn_pred_s_train)))

        # XGBoost RMSE for DeltaR
        xgb_pred_r_train = xgb_models['DeltaR'][asset].predict(X_train)
        rmse_r_xgb.append(sqrt(mean_squared_error(y_train[asset], xgb_pred_r_train)))

        # XGBoost RMSE for DeltaSigma
        xgb_pred_s_train = xgb_models['DeltaSigma'][asset].predict(X_train)
        rmse_s_xgb.append(sqrt(mean_squared_error(y_train[asset], xgb_pred_s_train)))

    # Convert lists to numpy arrays for vectorized operations
    rmse_r_nn = np.array(rmse_r_nn)
    rmse_r_xgb = np.array(rmse_r_xgb)
    rmse_s_nn = np.array(rmse_s_nn)
    rmse_s_xgb = np.array(rmse_s_xgb)

    # Compute weights inversely proportional to RMSE
    weights_r_nn = 1 / (rmse_r_nn + 1e-10)  # Add small value to prevent division by zero
    weights_r_xgb = 1 / (rmse_r_xgb + 1e-10)
    weights_s_nn = 1 / (rmse_s_nn + 1e-10)
    weights_s_xgb = 1 / (rmse_s_xgb + 1e-10)

    # Normalize weights so that they sum to 1 for each prediction type
    weights_r_total = weights_r_nn + weights_r_xgb
    weights_s_total = weights_s_nn + weights_s_xgb

    weights_r_nn /= weights_r_total
    weights_r_xgb /= weights_r_total
    weights_s_nn /= weights_s_total
    weights_s_xgb /= weights_s_total

    # Log the weights for verification
    print(f"Weights for DeltaR - Neural Networks: {weights_r_nn}")
    print(f"Weights for DeltaR - XGBoost: {weights_r_xgb}")
    print(f"Weights for DeltaSigma - Neural Networks: {weights_s_nn}")
    print(f"Weights for DeltaSigma - XGBoost: {weights_s_xgb}")

    # Collect and combine predictions using dynamic weights
    print("Collecting and combining predictions for each asset using dynamic weights...")
    for idx, asset in enumerate(price_data.columns):
        # Neural Network predictions
        nn_pred_r = nn_models['DeltaR'][asset].predict(X_test)[-1]  # DeltaR for the last test sample
        nn_pred_s = nn_models['DeltaSigma'][asset].predict(X_test)[-1]  # DeltaSigma for the last test sample

        # XGBoost predictions
        xgb_pred_r = xgb_models['DeltaR'][asset].predict(X_test)[-1]  # DeltaR
        xgb_pred_s = xgb_models['DeltaSigma'][asset].predict(X_test)[-1]  # DeltaSigma

        # Combine predictions using dynamic weights
        deltaR_final[idx] = (weights_r_nn[idx] * nn_pred_r) + (weights_r_xgb[idx] * xgb_pred_r)
        deltaSigma_final[idx] = (weights_s_nn[idx] * nn_pred_s) + (weights_s_xgb[idx] * xgb_pred_s)

    # Assign predictions to DataFrame for verification
    pred_df = pd.DataFrame({
        'DeltaR_Pred': deltaR_final,
        'DeltaSigma_Pred': deltaSigma_final
    }, index=price_data.columns)

    print("Combined Predictions after Dynamic Weighted Averaging:")
    print(pred_df)

    # Verify that predictions are not identical
    if np.all(deltaR_final == deltaR_final[0]) or np.all(deltaSigma_final == deltaSigma_final[0]):
        print("All predictions are identical. Please check the aggregation method.")
    else:
        # Optionally, normalize the targets before passing to GNN
        scaler_r = StandardScaler()
        scaler_s = StandardScaler()

        deltaR_final_scaled = scaler_r.fit_transform(deltaR_final.reshape(-1,1)).flatten()
        deltaSigma_final_scaled = scaler_s.fit_transform(deltaSigma_final.reshape(-1,1)).flatten()

        # Check uniqueness after scaling
        unique_deltaR_scaled = np.unique(deltaR_final_scaled)
        unique_deltaSigma_scaled = np.unique(deltaSigma_final_scaled)
        print(f"Unique values in scaled DeltaR: {unique_deltaR_scaled}")
        print(f"Unique values in scaled DeltaSigma: {unique_deltaSigma_scaled}")

        # Prepare GNN workflow with scaled targets
        print("Starting GNN workflow with dynamically weighted and scaled DeltaR and DeltaSigma...")
        gnn_results = full_gnn_workflow(price_data, deltaR_final_scaled, deltaSigma_final_scaled)
        print("GNN integration with Dynamic Weighted Ensemble completed successfully!")


In [ ]:
# =======================
#  CELL 4
#  Step 4: Implementing Advanced Optimization Methods with Constraints (Updated)
# =======================

import numpy as np
import pandas as pd
import logging
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
from math import sqrt

# Optimization Libraries
try:
    import pyswarms as ps
except ImportError:
    print("Installing pyswarms for Particle Swarm Optimization...")
    !pip install pyswarms
    import pyswarms as ps

# Define Upper and Lower Bounds for each asset
# Example: No asset can have less than 2% or more than 20% of the portfolio
# Adjust these values as per your requirements
lower_bounds = np.array([0.02] * price_data.shape[1])  # 2% lower bound
upper_bounds = np.array([0.20] * price_data.shape[1])  # 20% upper bound

# Define the objective function with penalty for constraint violations
def portfolio_performance_penalized(weights, returns, cov_matrix, risk_aversion=1.0, penalty_factor=1e6):
    """
    Calculate the portfolio performance with penalties for constraint violations.

    Args:
        weights (np.ndarray): Portfolio weights.
        returns (np.ndarray): Expected returns for each asset.
        cov_matrix (np.ndarray): Covariance matrix of asset returns.
        risk_aversion (float): Risk aversion coefficient.
        penalty_factor (float): Penalty factor for constraint violations.

    Returns:
        float: Penalized objective function value.
    """
    portfolio_return = np.dot(weights, returns)
    portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    objective = -(portfolio_return - risk_aversion * portfolio_risk)

    # Penalty for weights not summing to 1
    penalty = penalty_factor * (np.sum(weights) - 1) ** 2

    # Penalty for weights below lower bounds or above upper bounds
    penalty += penalty_factor * np.sum((weights < lower_bounds) * (lower_bounds - weights))
    penalty += penalty_factor * np.sum((weights > upper_bounds) * (weights - upper_bounds))

    return objective + penalty

# Define the constraint: Sum of weights equals 1
def weight_constraint(weights):
    return np.sum(weights) - 1

# Bounds for optimization (0 to 1 for each asset)
bounds_qp_sa = [(lb, ub) for lb, ub in zip(lower_bounds, upper_bounds)]

# Initial guess: Equal weights within bounds
initial_guess = np.array([max(1.0 / price_data.shape[1], lb) for lb in lower_bounds])
initial_guess /= np.sum(initial_guess)  # Normalize to sum to 1

# Extract returns and covariance from predictions
# Extract returns and covariance from predictions
expected_returns = deltaR_final_scaled  # Use scaled deltaR
covariance_matrix = np.diag(deltaSigma_final_scaled ** 2)  # Use scaled deltaSigma

# Risk aversion coefficient
lambda_risk = 1.0

# 1. Quadratic Programming (Mean-Variance Optimization) using SciPy's minimize
def optimize_portfolio_qp(expected_returns, covariance_matrix, risk_aversion=1.0):
    constraints = [{'type': 'eq', 'fun': weight_constraint}]
    result = minimize(portfolio_performance_penalized, initial_guess,
                      args=(expected_returns, covariance_matrix, risk_aversion),
                      method='SLSQP',
                      bounds=bounds_qp_sa,
                      constraints=constraints,
                      options={'maxiter': 1000, 'ftol': 1e-9})
    if result.success:
        optimized_weights = result.x
        # Enforce bounds manually (optional redundancy)
        optimized_weights = np.clip(optimized_weights, lower_bounds, upper_bounds)
        # Normalize weights to sum to 1
        optimized_weights /= np.sum(optimized_weights)
        portfolio_ret = np.dot(optimized_weights, expected_returns)
        portfolio_risk = np.sqrt(np.dot(optimized_weights.T, np.dot(covariance_matrix, optimized_weights)))
        print(f"QP Optimization Success: Return={portfolio_ret:.4f}, Risk={portfolio_risk:.4f}")
        return optimized_weights
    else:
        print("QP Optimization failed.")
        return None

# 2. Simulated Annealing (Using SciPy's minimize with SLSQP and penalties)
def optimize_portfolio_sa(expected_returns, covariance_matrix, risk_aversion=1.0):
    def objective(weights):
        return portfolio_performance_penalized(weights, expected_returns, covariance_matrix, risk_aversion)

    constraints_sa = [{'type': 'eq', 'fun': weight_constraint}]
    result = minimize(objective, initial_guess, method='SLSQP',
                      bounds=bounds_qp_sa,
                      constraints=constraints_sa,
                      options={'maxiter': 1000, 'ftol': 1e-9})
    if result.success:
        optimized_weights = result.x
        # Enforce bounds manually (optional redundancy)
        optimized_weights = np.clip(optimized_weights, lower_bounds, upper_bounds)
        # Normalize weights to sum to 1
        optimized_weights /= np.sum(optimized_weights)
        portfolio_ret = np.dot(optimized_weights, expected_returns)
        portfolio_risk = np.sqrt(np.dot(optimized_weights.T, np.dot(covariance_matrix, optimized_weights)))
        print(f"Simulated Annealing Success: Return={portfolio_ret:.4f}, Risk={portfolio_risk:.4f}")
        return optimized_weights
    else:
        print("Simulated Annealing Optimization failed.")
        return None

# 3. Particle Swarm Optimization using pyswarms with Penalty
def optimize_portfolio_pso(expected_returns, covariance_matrix, risk_aversion=1.0):
    # Define objective function for PSO (to minimize)
    def f(x):
        # x is a swarm of particles, each row is a position
        obj = np.array([portfolio_performance_penalized(w, expected_returns, covariance_matrix, risk_aversion) for w in x])
        return obj

    # PSO does not support constraints directly; use penalty method
    options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
    optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=price_data.shape[1],
                                        options=options, bounds=(lower_bounds, upper_bounds))

    best_cost, best_pos = optimizer.optimize(f, iters=200, verbose=False)

    # Enforce bounds manually
    best_pos = np.clip(best_pos, lower_bounds, upper_bounds)
    # Normalize weights to sum to 1
    best_pos /= np.sum(best_pos)

    portfolio_ret = np.dot(best_pos, expected_returns)
    portfolio_risk = np.sqrt(np.dot(best_pos.T, np.dot(covariance_matrix, best_pos)))
    print(f"PSO Optimization Success: Return={portfolio_ret:.4f}, Risk={portfolio_risk:.4f}")

    return best_pos

# Execute all optimization methods and store results
def execute_all_optimizations(expected_returns, covariance_matrix, risk_aversion=1.0):
    optim_results = {}

    # 1. Quadratic Programming
    optim_results['QP'] = optimize_portfolio_qp(expected_returns, covariance_matrix, risk_aversion)

    # 2. Simulated Annealing
    optim_results['SA'] = optimize_portfolio_sa(expected_returns, covariance_matrix, risk_aversion)

    # 3. Particle Swarm Optimization
    optim_results['PSO'] = optimize_portfolio_pso(expected_returns, covariance_matrix, risk_aversion)

    # You can add more optimization methods here following the same pattern

    return optim_results

# Example Usage
if 'expected_returns' not in globals() or 'covariance_matrix' not in globals():
    print("expected_returns or covariance_matrix not defined. Please ensure Step 2 and 3 are executed correctly.")
else:
    print("Executing all optimization methods with constraints...")
    optim_results = execute_all_optimizations(expected_returns, covariance_matrix, risk_aversion=lambda_risk)

    # Display the optimized weights and their performance
    for method, weights in optim_results.items():
        if weights is not None:
            portfolio_ret = np.dot(weights, expected_returns)
            portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
            print(f"{method} Optimization => Return: {portfolio_ret:.4f}, Risk: {portfolio_risk:.4f}")
            print(f"\n{method} Optimized Weights:")
            weights_df = pd.DataFrame(weights, index=price_data.columns, columns=['Weight'])
            print(weights_df)
        else:
            print(f"{method} Optimization did not return valid weights.")

    print("All optimization methods executed successfully with constraints!")


In [ ]:
# =======================
#  CELL 5 - Updated
#  Step 5: Iterative Partial-Derivative Layering Approach (Updated)
# =======================

import numpy as np
import pandas as pd
import logging
from sklearn.metrics import mean_squared_error
from math import sqrt

# Define convergence criteria
def check_convergence(old_ret, new_ret, threshold=1e-4):
    return np.abs(new_ret - old_ret) < threshold

# Iterative Optimization Function
def iterative_portfolio_optimization(price_data, initial_weights, deltaR, deltaSigma, covariance_matrix, risk_aversion=1.0, max_iterations=10, threshold=1e-4):
    """
    Iteratively optimize the portfolio by recalculating marginal contributions.

    Args:
        price_data (pd.DataFrame): Asset price data.
        initial_weights (np.ndarray): Initial portfolio weights.
        deltaR (np.ndarray): Marginal return contributions from ML models.
        deltaSigma (np.ndarray): Marginal risk contributions from ML models.
        covariance_matrix (np.ndarray): Covariance matrix.
        risk_aversion (float): Risk aversion coefficient.
        max_iterations (int): Maximum number of iterations.
        threshold (float): Convergence threshold.

    Returns:
        dict: Final optimized weights and performance metrics.
    """
    weights = initial_weights.copy()
    portfolio_ret, portfolio_risk = np.dot(weights, deltaR), np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    print(f"Initial Portfolio => Return: {portfolio_ret:.4f}, Risk: {portfolio_risk:.4f}")

    for iteration in range(1, max_iterations + 1):
        print(f"\n--- Iteration {iteration} ---")

        # Step 2: Optimize portfolio with updated deltaR and deltaSigma
        optim_results = execute_all_optimizations(deltaR, covariance_matrix, risk_aversion)

        # Select the method with the lowest RMSE between predicted and actual deltaR
        # Calculate RMSE for each method
        rmse_scores = {}
        for method, weights_opt in optim_results.items():
            if weights_opt is not None:
                predicted_deltaR = weights_opt * deltaR
                rmse = sqrt(mean_squared_error(deltaR, predicted_deltaR))
                rmse_scores[method] = rmse
            else:
                rmse_scores[method] = np.inf  # Assign infinity if optimization failed

        # Select the method with the lowest RMSE
        selected_method = min(rmse_scores, key=rmse_scores.get)
        new_weights = optim_results[selected_method]

        # Calculate new portfolio performance
        new_portfolio_ret = np.dot(new_weights, deltaR)
        new_portfolio_risk = np.sqrt(np.dot(new_weights.T, np.dot(covariance_matrix, new_weights)))
        print(f"{selected_method} Optimization => Return: {new_portfolio_ret:.4f}, Risk: {new_portfolio_risk:.4f}")
        print(f"Selected Method RMSE: {rmse_scores[selected_method]:.6f}")

        # Check for convergence
        if check_convergence(portfolio_ret, new_portfolio_ret, threshold):
            print("Convergence achieved.")
            weights = new_weights
            portfolio_ret, portfolio_risk = new_portfolio_ret, new_portfolio_risk
            break
        else:
            weights = new_weights
            portfolio_ret, portfolio_risk = new_portfolio_ret, new_portfolio_risk

    print("\n--- Iterative Optimization Completed ---")
    print(f"Final Portfolio => Return: {portfolio_ret:.4f}, Risk: {portfolio_risk:.4f}")
    print("Final Optimized Weights:")
    weights_df = pd.DataFrame(weights, index=price_data.columns, columns=['Weight'])
    print(weights_df)

    return {
        'weights': weights,
        'return': portfolio_ret,
        'risk': portfolio_risk
    }

# Example Usage
if 'optim_results' not in globals() or not optim_results:
    print("Optimization results not found. Please ensure Step 4 is executed correctly.")
else:
    # Select the best method based on RMSE
    # Calculate RMSE for each method
    rmse_scores = {}
    for method, weights_opt in optim_results.items():
        if weights_opt is not None:
            predicted_deltaR = weights_opt * deltaR_final_scaled
            rmse = sqrt(mean_squared_error(deltaR_final_scaled, predicted_deltaR))
            rmse_scores[method] = rmse
        else:
            rmse_scores[method] = np.inf  # Assign infinity if optimization failed

    # Select method with lowest RMSE
    best_method = min(rmse_scores, key=rmse_scores.get)
    best_weights = optim_results[best_method]
    print(f"Selected Best Method for Iterative Optimization: {best_method} with RMSE={rmse_scores[best_method]:.6f}")

    # Execute iterative optimization
    final_portfolio = iterative_portfolio_optimization(
        price_data,
        best_weights,
        deltaR_final_scaled,
        deltaSigma_final_scaled,
        covariance_matrix,
        risk_aversion=lambda_risk,
        max_iterations=10,
        threshold=1e-4
    )

    print("Iterative Partial-Derivative Layering Approach completed successfully!")


In [ ]:
# =======================
#  CELL 6
#  Step 6: Finalizing ACHL Integration with Advanced Performance Metrics
# =======================

import numpy as np
import pandas as pd
import logging
from scipy.stats import linregress
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from math import sqrt

# Configure logging if not already done
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define Performance Metrics Functions

def sortino_ratio(returns, target=0.0):
    """
    Calculate the Sortino Ratio.

    Args:
        returns (np.ndarray or pd.Series): Portfolio returns.
        target (float): Target or minimum acceptable return.

    Returns:
        float: Sortino Ratio.
    """
    excess_returns = returns - target
    downside_returns = excess_returns[excess_returns < 0]
    downside_std = np.sqrt(np.mean(downside_returns ** 2))
    if downside_std == 0:
        return np.nan
    return np.mean(excess_returns) / downside_std

def calculate_alpha_beta(portfolio_returns, benchmark_returns):
    """
    Calculate Alpha and Beta of the portfolio against a benchmark.

    Args:
        portfolio_returns (pd.Series): Portfolio returns.
        benchmark_returns (pd.Series): Benchmark returns.

    Returns:
        tuple: (Alpha, Beta)
    """
    slope, intercept, r_value, p_value, std_err = linregress(benchmark_returns, portfolio_returns)
    beta = slope
    alpha = intercept
    return alpha, beta

def information_ratio(portfolio_returns, benchmark_returns):
    """
    Calculate the Information Ratio of the portfolio.

    Args:
        portfolio_returns (pd.Series): Portfolio returns.
        benchmark_returns (pd.Series): Benchmark returns.

    Returns:
        float: Information Ratio.
    """
    active_returns = portfolio_returns - benchmark_returns
    tracking_error = np.std(active_returns)
    if tracking_error == 0:
        return np.nan
    return np.mean(active_returns) / tracking_error

# Benchmark Portfolios

def get_benchmark_returns(price_data, expected_returns, covariance_matrix):
    """
    Calculate benchmark returns using various portfolio strategies.

    Args:
        price_data (pd.DataFrame): Asset price data.
        expected_returns (np.ndarray): Expected returns vector.
        covariance_matrix (np.ndarray): Covariance matrix.

    Returns:
        tuple: (dict of benchmark returns, dict of benchmark weights)
    """
    # 1. Equal-Weighted Portfolio
    equal_weights = np.array([1.0 / price_data.shape[1]] * price_data.shape[1])
    equal_returns = calculate_portfolio_returns(equal_weights, price_data)

    # 2. Minimum Variance Portfolio (Efficient Frontier)
    def minimum_variance_portfolio(expected_returns, cov_matrix):
        num_assets = len(expected_returns)
        args = (cov_matrix,)
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bounds_mv = tuple((0, 1) for asset in range(num_assets))
        initial_guess_mv = np.array([1.0 / num_assets] * num_assets)
        result = minimize(portfolio_variance, initial_guess_mv, args=args,
                          method='SLSQP', bounds=bounds_mv, constraints=constraints)
        if result.success:
            return result.x
        else:
            print("Minimum Variance Portfolio Optimization failed.")
            return equal_weights  # Fallback to Equal-Weighted

    def portfolio_variance(weights, cov_matrix):
        return np.dot(weights.T, np.dot(cov_matrix, weights))

    min_var_weights = minimum_variance_portfolio(expected_returns, covariance_matrix)
    min_var_returns = calculate_portfolio_returns(min_var_weights, price_data)


    # 3. Sparse Portfolio using Lasso Regression
    # Reshape expected_returns to match the number of samples in price_data
    expected_returns_reshaped = np.tile(expected_returns, (price_data.shape[0], 1))

    lasso = Lasso(alpha=0.1)
    lasso.fit(price_data, expected_returns_reshaped)  # Use reshaped expected returns

    # Modification to ensure correct shape for sparse_weights
    sparse_weights = np.abs(lasso.coef_)
    sparse_weights = sparse_weights.mean(axis=0)  # Averaging weights across time
    sparse_weights = sparse_weights / np.sum(sparse_weights) if np.sum(sparse_weights) != 0 else np.array([1.0 / price_data.shape[1]] * price_data.shape[1])

    # Reshape to (20,) if not already in that shape
    if sparse_weights.shape != (price_data.shape[1],):
      sparse_weights = sparse_weights.reshape(price_data.shape[1],)

    sparse_returns = calculate_portfolio_returns(sparse_weights, price_data)


    benchmark_weights = {
        'Equal-Weighted': equal_weights,
        'Minimum-Variance': min_var_weights,
        'Sparse': sparse_weights
    }

    benchmark_returns = {
        'Equal-Weighted': equal_returns,
        'Minimum-Variance': min_var_returns,
        'Sparse': sparse_returns
    }

    return benchmark_returns, benchmark_weights

# Calculate Portfolio Returns

def calculate_portfolio_returns(weights, price_data):
    """
    Calculate the portfolio returns given weights and price data.

    Args:
        weights (np.ndarray): Portfolio weights.
        price_data (pd.DataFrame): Asset price data.

    Returns:
        pd.Series: Portfolio returns.
    """
    portfolio_prices = price_data.dot(weights)
    portfolio_returns = portfolio_prices.pct_change().dropna()
    return portfolio_returns

# Calculate Performance Metrics

def calculate_performance_metrics(portfolio_returns, benchmark_returns):
    """
    Calculate performance metrics for the portfolio.

    Args:
        portfolio_returns (pd.Series): Portfolio returns.
        benchmark_returns (pd.Series): Benchmark returns.

    Returns:
        dict: Performance metrics.
    """
    sortino = sortino_ratio(portfolio_returns)
    alpha, beta = calculate_alpha_beta(portfolio_returns, benchmark_returns)
    info_ratio = information_ratio(portfolio_returns, benchmark_returns)

    metrics = {
        'Sortino Ratio': sortino,
        'Alpha': alpha,
        'Beta': beta,
        'Information Ratio': info_ratio
    }
    return metrics

# Finalize Performance Metrics for All Portfolios

def finalize_portfolio_performance(achl_weights, price_data, expected_returns, covariance_matrix, risk_aversion=1.0):
    """
    Calculate and log advanced performance metrics for the ACHL and benchmark portfolios.

    Args:
        achl_weights (np.ndarray): Optimized ACHL portfolio weights.
        price_data (pd.DataFrame): Asset price data.
        expected_returns (np.ndarray): Expected returns vector.
        covariance_matrix (np.ndarray): Covariance matrix.
        risk_aversion (float): Risk aversion coefficient.

    Returns:
        pd.DataFrame: Performance metrics for all portfolios.
    """
    performance_metrics = {}

    # 1. ACHL Portfolio
    achl_returns = calculate_portfolio_returns(achl_weights, price_data)
    performance_metrics['ACHL'] = {}
    performance_metrics['ACHL']['Return'] = np.mean(achl_returns)
    performance_metrics['ACHL']['Risk'] = np.std(achl_returns)
    performance_metrics['ACHL']['Sortino Ratio'] = sortino_ratio(achl_returns)

    # 2. Benchmark Portfolios
    benchmark_returns_dict, benchmark_weights_dict = get_benchmark_returns(price_data, expected_returns, covariance_matrix)

    # For Alpha and Beta, define a common market benchmark. Here, we'll use the Minimum-Variance Portfolio as a proxy.
    # Replace this with an actual market index for more accurate results.
    market_returns = benchmark_returns_dict['Minimum-Variance']

    for name, returns in benchmark_returns_dict.items():
        sortino = sortino_ratio(returns)
        alpha, beta = calculate_alpha_beta(returns, market_returns)
        info_ratio = information_ratio(returns, market_returns)

        performance_metrics[name] = {
            'Return': np.mean(returns),
            'Risk': np.std(returns),
            'Sortino Ratio': sortino,
            'Alpha': alpha,
            'Beta': beta,
            'Information Ratio': info_ratio
        }

    # 3. ACHL Performance Metrics
    achl_alpha, achl_beta = calculate_alpha_beta(achl_returns, market_returns)
    achl_info_ratio = information_ratio(achl_returns, market_returns)

    performance_metrics['ACHL']['Alpha'] = achl_alpha
    performance_metrics['ACHL']['Beta'] = achl_beta
    performance_metrics['ACHL']['Information Ratio'] = achl_info_ratio

    # Create a DataFrame for easy comparison
    metrics_df = pd.DataFrame(performance_metrics).T
    print("\n=== Summary of Performance Metrics ===")
    print(metrics_df)

    return metrics_df

# Example Usage

if 'final_portfolio' not in globals():
    print("Final portfolio not found. Please ensure Step 5 is executed correctly.")
else:
    # Calculate ACHL Portfolio Returns
    achl_weights = final_portfolio['weights']
    achl_returns = calculate_portfolio_returns(achl_weights, price_data)

    # Get Benchmark Returns
    benchmark_returns_dict, benchmark_weights_dict = get_benchmark_returns(price_data, expected_returns, covariance_matrix)

    # Calculate and log performance metrics
    print("Finalizing performance metrics for ACHL and Benchmark Portfolios...")
    performance_df = finalize_portfolio_performance(
        achl_weights,
        price_data,
        expected_returns,
        covariance_matrix,
        risk_aversion=lambda_risk
    )

    print("Advanced performance metrics calculation completed successfully!")


In [ ]:
# =======================
#  CELL 7
#  Step 7: Visualization of Portfolio Performance
# =======================

import matplotlib.pyplot as plt

def plot_cumulative_returns(price_data, weights_dict, title='Cumulative Returns Comparison'):
    """
    Plot cumulative returns of different portfolios.

    Args:
        price_data (pd.DataFrame): Asset price data.
        weights_dict (dict): Dictionary with portfolio names as keys and weights as values.
        title (str): Title of the plot.
    """
    plt.figure(figsize=(12, 8))

    for name, weights in weights_dict.items():
        returns = calculate_portfolio_returns(weights, price_data)
        cumulative_returns = (1 + returns).cumprod()
        plt.plot(cumulative_returns, label=name)

    plt.xlabel('Date')
    plt.ylabel('Cumulative Returns')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

# Prepare weights dictionary including ACHL and benchmarks
weights_dict = {
    'ACHL': achl_weights,
    'Equal-Weighted': benchmark_weights_dict['Equal-Weighted'],
    'Minimum-Variance': benchmark_weights_dict['Minimum-Variance'],
    'Sparse': benchmark_weights_dict['Sparse']
}

# Plot cumulative returns
plot_cumulative_returns(price_data, weights_dict, title='Cumulative Returns Comparison: ACHL vs Benchmarks')
